In [0]:
import cv2
import numpy as np


img = cv2.imread('mountain1.jpg')
gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
sift = cv2.xfeatures2d.SIFT_create()
#kp = sift.detect(gray,None)
kp, des = sift.detectAndCompute(gray,None)
img=cv2.drawKeypoints(gray,kp,None) 
cv2.imwrite('task1_sift1.jpg',img)

True

In [0]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('mountain1.jpg',0)          # queryImage
img2 = cv2.imread('mountain2.jpg',0) # trainImage

# Initiate SIFT detector
#sift = cv2.SIFT()
sift = cv2.xfeatures2d.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params,search_params)

matches = flann.knnMatch(des1,des2,k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.75*n.distance:
        matchesMask[i]=[1,0]

draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = 0)

img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
cv2.imwrite('task1_matches_knn.jpg',img3)
#plt.imshow(img3,),plt.show()

True

In [0]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from numpy import float32, int32
MIN_MATCH_COUNT = 32

UBIT = 'rohanhem'
np.random.seed(sum([ord(c) for c in UBIT]))


def warp(img1, img2, H):
    
    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    pts1 = float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
    pts2 = float32([[0,0],[0,h2],[w2,h2],[w2,0]]).reshape(-1,1,2)
    pts2_ = cv2.perspectiveTransform(pts2, H)
    pts = np.concatenate((pts1, pts2_), axis=0)
    [xmin, ymin] = int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = int32(pts.max(axis=0).ravel() + 0.5)
    t = [-xmin,-ymin]
    Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) 
    result = cv2.warpPerspective(img1, Ht.dot(H), (xmax-xmin, ymax-ymin))
    result[t[1]:h1+t[1],t[0]:w1+t[0]] = img2
    return result

img1 = cv2.imread('mountain1.jpg',1)          # queryImage
img2 = cv2.imread('mountain2.jpg',1) # trainImage
cv2.imwrite('qwer.jpg',img1)

sift = cv2.xfeatures2d.SIFT_create()

kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1,des2,k=2)

good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append(m)
        
        
if len(good)>MIN_MATCH_COUNT:
    dst_pts = float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    src_pts = float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    im_out = warp(img1, img2, M)
    matchesMask = mask.ravel().tolist()


else:
    print("Not enough matches are found - %d/%d" %(len(good),MIN_MATCH_COUNT))
    matchesMask = None
    
np.random.shuffle(matchesMask)
matchesMask = matchesMask[:10]  

draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

np.random.shuffle(good)
good = good[:10]
img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
print (M)
cv2.imwrite('task1_matches.jpg',img3)
cv2.imwrite('task1_pano.jpg',im_out)
#plt.imshow(img3, 'gray'),plt.show()    

[[ 5.90587029e-01  1.32589412e-01  2.58484863e+02]
 [-2.82834995e-01  8.60847918e-01  5.19136415e+01]
 [-7.32943586e-04 -1.04200571e-04  1.00000000e+00]]


True